In [1]:
import os, librosa, numpy as np, pandas as pd, statistics, scipy, re, pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings('ignore')

""efficient implementation approach" cf. some paper

In [35]:
os.chdir(r"/content/drive/MyDrive/Tracks")
df = pd.read_excel(r"/content/drive/MyDrive/You_S1_Data_NoBillboardRanking.xlsx")
df = df.sort_values(by=['uniq_id'])
df = df.reset_index(drop=True)
df.head(5)

,uniq_id,date,month,year,d2014,d2015,d2016,q1_14,q2_14,q3_14,...,way,well,were,what,when,where,who,will,with,yeah
0,1.0,2014-02-08,2.0,2014.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
1,2.0,2014-02-08,2.0,2014.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,3.0,2014-02-08,2.0,2014.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,4.0,2014-02-08,2.0,2014.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,5.0,2014-02-08,2.0,2014.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [36]:
#Cast Song Title from float (to int) to string
floatRowsMask = np.where([isinstance(x, float) for x in  df.song])
df.song.iloc[floatRowsMask] = df.loc[floatRowsMask, "song"].astype('int').astype('str')

In [37]:
#Rename all files in Tracks folder to lower case
path = "/content/drive/MyDrive/Tracks/"
for file in os.listdir(path):
    os.rename(path + file, path + file.lower())

In [38]:
for x in range(len(df.date)): #2022QuickFix
    song_temp = df["song"][x] 
    song_temp = re.sub("[/:\"]", "", song_temp)
    song_temp = re.sub("[/:\"]", "", song_temp)
    song_temp = re.sub("[\?\*]", "#", song_temp)
    song_temp = re.sub("腎", "t", song_temp)
    song_temp = song_temp.lower()
    if song_temp.find("(") != -1: song_temp = song_temp[:song_temp.find("(")-1] + song_temp[song_temp.find(")")+1:] 
    df["song"][x] = re.sub("ac/dc", "acdc", song_temp)
    artist_temp = df["artist"][x] 
    artist_temp = re.sub("[/:\"]", "", artist_temp)
    artist_temp = re.sub("[\?\*]", "#", artist_temp)
    artist_temp = artist_temp.lower()
    df["artist"][x] = re.sub("ac/dc", "acdc", artist_temp)

##Control loop to check if error occurs (ie. song not found in folder)
# for x in range(len(df.date)):
#     b = df["song"][x] #2022QuickFix
#     if b.find("(") != -1: 
#       b = b[:b.find("(")-1] + b[b.find(")")+1:]  #2022QuickFix
#       df["song"][x] = b
#     a = b + " - " + df["artist"][x]  
#     if a.lower() in [x[:-4] for x in os.listdir()]:
#         pass
#     else:
#         print(a)
#         print(df["artist_song"][x])

In [ ]:
j_chroma = range(3, 9)
hop_length = 512
def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))
def d(s1, s2):
    M = (s1 + s2) / 2
    return (kl_divergence(s1, M) + kl_divergence(s2, M)) / 2
def s_eff(a,b):
    return matrixC[b] - matrixC[a]

def eucl_dist(a,b):
    return np.linalg.norm(a-b)

scChromaOUT = []
for ii in range(len(df.date)):
    if df.song[ii] not in [x[0] for x in scChromaOUT]:
        print(df.song[ii],ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]))
        chromagram = pd.DataFrame(librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length, )).transpose()
        matrixC = []
        for i in range(len(chromagram[1])):
            matrixC.append(sum(np.array(chromagram.iloc[0:i+1])))
        scChroma = []
        v_eff=[]
        for j in j_chroma:
            w = 2 ** (j - 1)
            v_eff = []
            for i in range(len(chromagram[1])):
                if i>w and i<len(chromagram[1])-w:
                    v_eff.append(d(s_eff(i - w + 1, i - 1), s_eff(i, i + w)))
                else:
                    v_eff.append(0)
            scChroma.append(statistics.mean(v_eff))
        scChromaOUT.append((df.song[ii],df.artist[ii],scChroma))

oceans 0
3
4
5
6
7
8
beautiful day 1
3
4
5
6
7
8
overcomer 2
3
4
5
6
7
8
this is amazing grace 3
3
4
5
6
7
8
shake 4
3
4
5
6
7
8
write your story 5
3
4
5
6
7
8
10,000 reasons 6
3
4
5
6
7
8
lord i need you 7
3
4
5
6
7
8
how to save a life 8
3
4
5
6
7
8
all you've ever wanted 9
3
4
5
6
7
8
keep making me 10
3
4
5
6
7
8
i can only imagine 11
3
4
5
6
7
8
whom shall i fear 12
3
4
5
6
7
8
love alone is worth the fight 13
3
4
5
6
7
8
i am 14
3
4
5
6
7
8
monster 15
3
4
5
6
7
8
you found me 16
3
4
5
6
7
8
thrive 17
3
4
5
6
7
8
the only name 18
3
4
5
6
7
8
difference maker 19
3
4
5
6
7
8
the heart 20
3
4
5
6
7
8
jesus, take the wheel 21
3
4
5
6
7
8
god's not dead 22
3
4
5
6
7
8
hello, my name is 23
3
4
5
6
7
8
speak life 24
3
4
5
6
7
8
redeemed 25
3
4
5
6
7
8
good morning 26
3
4
5
6
7
8
hero 27
3
4
5
6
7
8
gold 28
3
4
5
6
7
8
broken hallelujah 29
3
4
5
6
7
8
not gonna die 30
3
4
5
6
7
8
this is now 31
3
4
5
6
7
8
worn 32
3
4
5
6
7
8
you and me 33
3
4
5
6
7
8
our god 34
3
4
5
6
7
8
i don't deserv

In [34]:
#with open('/content/drive/MyDrive/scChromaOUT1.pkl', 'wb') as f:
#  pickle.dump(scChromaOUT, f)

with open('/content/drive/MyDrive/scChromaOUT1.pkl', 'rb') as f:
   scChromaOUT = pickle.load(f)

In [48]:
#[x for x in range(len(scChromaOUT)) if "mean" in scChromaOUT[x][0]]

[568, 633, 697, 944, 1245, 1695]

In [54]:
#scChromaOUT[1695]

('wat u mean',
 'dae dae',
 [1.747603797549854,
  1.9804192044687103,
  2.3437729975855706,
  2.9493167563332943,
  3.873596307142461,
  6.491969528531878])

In [32]:
##scChromaOUT[105] = ('castaway',
# 'zac brown band ',
# [1.5286225971986438,
#  1.9564730549538154,
#  2.9522507891120457,
#  4.643816268279218,
#  6.6410810748934965,
#  6.702313356935985])

In [39]:
df["Chroma1"],df["Chroma2"],df["Chroma3"],df["Chroma4"],df["Chroma5"],df["Chroma6"]=0.0,0.0,0.0,0.0,0.0,0.0

for x in range(len(df.date)):
    try:
        el=[y for y in scChromaOUT if y[0]==df.song[x]][0][-1]
        df["Chroma1"][x],df["Chroma2"][x],df["Chroma3"][x],df["Chroma4"][x],df["Chroma5"][x],df["Chroma6"][x]=el
    except:
        print(df.song[x])

wat u meanwat u mean (aye, aye, aye


In [57]:
#df.Chroma1.loc[1693:1697]

1693    1.385494
1694    1.718914
1695    1.531141
1696    1.519318
1697    1.778577
Name: Chroma1, dtype: float64

In [58]:
#np.where(df.Chroma1==0.0)

(array([3737]),)

In [59]:
#df.song[3737]

'wat u meanwat u mean (aye, aye, aye'

In [61]:
df["Rank"] = np.tile(np.arange(50, 0, -1), 84)

In [62]:
import statsmodels.api as sm
my_cols = ["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.879
Date:                Mon, 05 Dec 2022   Prob (F-statistic):            0.00841
Time:                        16:55:32   Log-Likelihood:                -17162.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4193   BIC:                         3.438e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.6415      3.774      5.469      0.0

In [ ]:
import time

#timbre and arousal
sr=44100
win_len = round(sr * 2.97)
win = np.hamming(win_len)
j_timbre = range(1, 7)

scTimbreOUT = []
for ii in range(len(df.date)):
    if df.song[ii] not in [x[0] for x in scTimbreOUT]:
        start_time = time.time()
        print(df.song[ii],ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]),sr)
        d = round(librosa.get_duration(x, sr=sr))  # in seconds, COME CONSIDERO IL ROUNDING?!?!?!?
        i, ls = 0, []
        while i < d * sr:
            a = x[i:i + win_len]  # win_len = datapoints in that window interval, +i <-- il movimento di 1 sec, (+0,+44100, +44100*2, ecc)
            if len(x[i:i + win_len]) < win_len: #per l'ultimo pezzo magari non lungo abbastanza
                a = np.array(list(x[i:i + win_len]) + [0 for x in range(win_len - len(x[i:i + win_len]))])
            windowed_segment = win * a
            ls_t = []
            for y in range(0, len(windowed_segment), 512):
                ls_t.append([x[0] for x in librosa.feature.mfcc(windowed_segment[y:y + 512], sr=sr, n_mfcc=36).tolist()]) #mettere lol[0:x con x>len(lol)] funge lo stesso
            df1 = pd.DataFrame(ls_t).transpose()
            ls.append([statistics.mean(df1.iloc[x]) for x in range(len(df1[35]))])
            #print((d*sr - i)//sr)
            i += sr
        df1 = pd.DataFrame(ls)

        matrixC = []
        for i in range(len(df1[35])):
            matrixC.append(sum(np.array(df1.iloc[0:i+1])))
        scTimbre = []
        v_eff=[]
        for j in j_timbre:
            w = 2 ** (j - 1)
            v_eff = []
            print(j)
            for i in range(len(df1[35])):
                if i>w and i<len(df1[35])-w:
                    v_eff.append(eucl_dist(s_eff(i - w + 1, i - 1), s_eff(i, i + w))) #kl div serve per le prob distro, non è quello che cerco, uso la eucledian distance https://groups.google.com/g/librosa/c/9Xs15kDImOg
                else:
                    v_eff.append(0)
            scTimbre.append(statistics.mean(v_eff))
        scTimbreOUT.append((df.song[ii],df.artist[ii],scTimbre))
        print("--- %s seconds ---" % (time.time() - start_time))

#
sr=44100
win_len = round(sr * 2.97)
win = np.hamming(win_len)
arousalOUT = []
for ii in range(len(df.date)):
    #if df.song[ii] not in [x[0] for x in arousalOUT] and df.artist[ii] not in [x[1] for x in arousalOUT]:
    if (df.song[ii], df.artist[ii]) not in [(x[0],x[1]) for x in arousalOUT]:
        #for el in os.listdir(os.getcwd()): #togli [:1] lool ma che cazzo avevo fatto
        print(df.song[ii], ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii], df.artist[ii]), sr)
        d = round(librosa.get_duration(x, sr=44100))  # in seconds, COME CONSIDERO IL ROUNDING?!?!?!?
        i, arousal = 0, []
        while i < d * sr:
            a = x[i:i + win_len]  # win_len = datapoints in that window interval, +i <-- il movimento di 1 sec, (+0,+44100, +44100*2, ecc)
            if len(x[i:i + win_len]) < win_len:
                a = np.array(list(x[i:i + win_len]) + [0 for x in range(win_len - len(x[i:i + win_len]))])
            windowed_segment = win * a
            arousal.append(np.sum(np.abs(windowed_segment)))
            #asaprint(d*sr - i)
            i += sr
        arousalOUT.append((df.song[ii],df.artist[ii],statistics.mean(arousal),statistics.stdev(arousal)))

Oceans 0
1
2
3
4
5
6
--- 552.8081092834473 seconds ---
Beautiful Day 1


KeyboardInterrupt: ignored

In [ ]:
# aa=[x for x in df.song]
# aa=list(set(aa))
# aaa=[x[0] for x in scChromaOUT]
# aaaa=[x for x in aa if x not in aaa]

# for ii in range(len(df.date)):
#     print(ii)
#     try:
#         if df.song[ii] not in [x[0] for x in scChromaOUT]:
#             print(df.song[ii],ii)
#             x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]))
#             chromagram = pd.DataFrame(librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length, )).transpose()
#             matrixC = []
#             for i in range(len(chromagram[1])): #quell'[1] che sarebbe la prima corda conta solo per il range!
#                 matrixC.append(sum(np.array(chromagram.iloc[0:i+1])))
#             scChroma = []
#             v_eff=[]
#             for j in j_chroma:
#                 w = 2 ** (j - 1)
#                 v_eff = []
#                 print(j)
#                 for i in range(len(chromagram[1])):
#                     #print(i)
#                     if i>w and i<len(chromagram[1])-w: #l'[1] conta sempre e solo per il range, le corde le faccio tutte
#                         v_eff.append(d(s_eff(i - w + 1, i - 1), s_eff(i, i + w)))
#                     else:
#                         v_eff.append(0)
#                 scChroma.append(statistics.mean(v_eff))
#             scChromaOUT.append((df.song[ii],df.artist[ii],scChroma))
#     except:
#         print("bababa")


# b=[]
# c=[] #quelli da rifare + quelli che ho messo per la prima volta in b!!!!!!!!!!
# bb=list(set([(df.song[x],df.artist[x]) for x in range(len(df.date))]))
# for x in bb:
#     if x[0] not in [x[0] for x in b]:
#         b.append(x)
#     else:
#         c.append(x)

# for x in b:
#     if x[0] in [x[0] for x in c]:
#         c.append(x)

# temp = []
# for x in ale:
#     if x[0] not in [x[0] for x in temp]:
#         temp.append(x)
#     else:
#          if isinstance([y[1] for y in temp if y[0]==x[0]][0],list):
#             temp.remove([y[1] for y in temp if y[0]==x[0]][0])
#             temp.append(x)

# for el in range(len(c)):
# #for i in list(len(df.date)):
#     print(el)
#     x, sr = librosa.load("{} - {}.mp3".format(c[el][0],c[el][1]))
#     chromagram = pd.DataFrame(librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length, )).transpose()
#     matrixC = []
#     for i in range(len(chromagram[1])): #quell'[1] che sarebbe la prima corda conta solo per il range!
#         matrixC.append(sum(np.array(chromagram.iloc[0:i+1])))
#     scChroma = []
#     v_eff=[]
#     for j in j_chroma:
#         w = 2 ** (j - 1)
#         v_eff = []
#         print(j)
#         for i in range(len(chromagram[1])): 
#             #print(i)
#             if i>w and i<len(chromagram[1])-w: #l'[1] conta sempre e solo per il range, le corde le faccio tutte
#                 v_eff.append(d(s_eff(i - w + 1, i - 1), s_eff(i, i + w)))
#             else:
#                 v_eff.append(0)
#         scChroma.append(statistics.mean(v_eff))
#     ale.append((c[el][0],c[el][1],scChroma))

# fix=[(c[-i],ale[i]) for i in range(-283,1,1)]
# ale=ale[:-284]
# for x in fix:
#     ale.append(x)

# df["Chroma1"],df["Chroma2"],df["Chroma3"],df["Chroma4"],df["Chroma5"],df["Chroma6"]=0.0,0.0,0.0,0.0,0.0,0.0

# for x in range(len(df.date)):
#     try:
#         el=[y for y in ale if y[0]==df.song[x]][0][-1]
#         df["Chroma1"][x],df["Chroma2"][x],df["Chroma3"][x],df["Chroma4"][x],df["Chroma5"][x],df["Chroma6"][x]=el
#     except:
#         print(df.song[x])


# #import pickle
# #with open(os.getcwd()[:-6]+'done2.pkl', 'wb') as f:
# #    pickle.dump(scChromaOUT, f)

# #with open(os.getcwd()[:-6]+'done2.pkl', 'rb') as f:
# #   ale = pickle.load(f)


In [ ]:
#main 

#main 

# efficient implementation approach (py is not c++ ;)
import os, librosa, numpy as np, pandas as pd, statistics, scipy

import matplotlib.pyplot as plt

df = pd.read_excel(r"C:\Users\alessio\Desktop\afterChroma.xlsx", engine='openpyxl')
os.chdir(r"C:\Users\alessio\Documents\Tracks")

def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))
def d(s1, s2):
    M = (s1 + s2) / 2
    return (kl_divergence(s1, M) + kl_divergence(s2, M)) / 2
def s_eff(a, b):
    return matrixC[b] - matrixC[a]
def eucl_dist(a, b):
    return np.linalg.norm(a - b)


import time

# timbre and arousal
sr = 44100
win_len = round(sr * 2.97)
win = np.hamming(win_len)
j_timbre = range(1, 7)

#scTimbreOUT = []
for ii in range(len(df.date)): #576,994, 2550
    print(ii)
    if (df.song[ii], df.artist[ii]) not in [(x[0], x[1]) for x in scTimbreOUT]:
        print(df.song[ii],ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]),sr)
        d = round(librosa.get_duration(x, sr=sr))  # in seconds, COME CONSIDERO IL ROUNDING?!?!?!?
        i, ls = 0, []
        while i < d * sr:
            a = x[i:i + win_len]  # win_len = datapoints in that window interval, +i <-- il movimento di 1 sec, (+0,+44100, +44100*2, ecc)
            if len(x[i:i + win_len]) < win_len: #per l'ultimo pezzo magari non lungo abbastanza
                a = np.array(list(x[i:i + win_len]) + [0 for x in range(win_len - len(x[i:i + win_len]))])
            windowed_segment = win * a
            ls_t = []
            for y in range(0, len(windowed_segment), 512):
                ls_t.append([x[0] for x in librosa.feature.mfcc(windowed_segment[y:y + 512], sr=sr, n_mfcc=36).tolist()]) #mettere lol[0:x con x>len(lol)] funge lo stesso
            df1 = pd.DataFrame(ls_t).transpose()
            ls.append([statistics.mean(df1.iloc[x]) for x in range(len(df1[35]))])
            #print((d*sr - i)//sr)
            i += sr
        df1 = pd.DataFrame(ls)

        matrixC = []
        for i in range(len(df1[35])):
            matrixC.append(sum(np.array(df1.iloc[0:i+1])))
        scTimbre = []
        v_eff=[]
        for j in j_timbre:
            w = 2 ** (j - 1)
            v_eff = []
            print(j)
            for i in range(len(df1[35])):
                if i>w and i<len(df1[35])-w:
                    v_eff.append(eucl_dist(s_eff(i - w + 1, i - 1), s_eff(i, i + w))) #kl div serve per le prob distro, non è quello che cerco, uso la eucledian distance https://groups.google.com/g/librosa/c/9Xs15kDImOg
                else:
                    v_eff.append(0)
            scTimbre.append(statistics.mean(v_eff))
        scTimbreOUT.append((df.song[ii],df.artist[ii],scTimbre))

import pickle

with open(r"C:\Users\alessio\Documents\mfcc.pkl", 'wb') as f:
    pickle.dump(mfcc, f)

with open(r"C:\Users\alessio\Documents\arousalOUT.pkl", 'rb') as f:
   arousalOUT = pickle.load(f)

# MFCC
import os, librosa, numpy as np, pandas as pd, statistics, scipy
df = pd.read_excel(r"C:\Users\alessio\Downloads\afterAll.xlsx", engine='openpyxl')
os.chdir(r"C:\Users\alessio\Documents\Tracks")
from sklearn.cluster import KMeans
sr = 44100
win_len = round(sr * 0.025)

mfcc = []
for ii in range(len(df.date)):
    print(ii)
    if (df.song[ii], df.artist[ii]) not in [(x[0], x[1]) for x in mfcc]:
        print(df.song[ii],ii)
        x, sr = librosa.load("{} - {}.mp3".format(df.song[ii],df.artist[ii]),sr)
        i, ls = 0, []
        d = round(librosa.get_duration(x, sr=sr))
        while i < d * sr:
           windowed_segment = x[i:i + win_len]  # win_len = datapoints in that window interval, +i <-- il movimento di 1 sec, (+0,+44100, +44100*2, ecc)
           if len(x[i:i + win_len]) < win_len:
               windowed_segment = np.array(
                   list(x[i:i + win_len]) + [0.0 for x in range(win_len - len(x[i:i + win_len]))])
           ls.append(librosa.feature.mfcc(windowed_segment, sr=sr, n_mfcc=20))
           i += round(0.015 * sr)

        llss = []
        for x in range(len(ls)):
           for y in range(20):
               llss.append(ls[x][y][0]), llss.append(ls[x][y][1]), llss.append(ls[x][y][2])

        i, final = 0, []
        while i < len(llss):
           final.append([llss[x + i] for x in range(1, 60, 3)])
           final.append([llss[x + i] for x in range(2, 60, 3)])
           final.append([llss[x + i] for x in range(3, 60, 3)])
           i += 60

        df1 = pd.DataFrame(final)
        mfcc.append((df.song[ii], df.artist[ii],df.genre[ii],[np.mean(df1[x]) for x in df1.columns]))  # credo che io debba fare l'average, il df di una singola canzone a 55k rows, se ce ne metto altre 4199 --> 230945000

'''
dovrai fare knn?? per genere, pero contando tutte e 4200 le righe, non solo 1800, ripartendo quindi dal dataset lo fai co list comprehension.
'''
import pickle
with open(r"C:\Users\alessio\Desktop\mfcc.pkl", 'rb') as f:
   mfcc = pickle.load(f)

df["toDropMFCC"]="Zero"
for x in range(len(df.date)):
    try:
        el = [y for y in mfcc if y[0]==df.song[x] and y[1]==df.artist[x]][0][-1] # and y[1]==df.artist[x]
        df["toDropMFCC"][x] = el
    except:
        print(df.song[x])
#
from sklearn.cluster import KMeans
mfccOUT=[]
for g in list(set([x[2] for x in mfcc])):
    kmeans = KMeans(n_clusters=32, random_state=42).fit(list(df.iloc[[x for x in range(len(df.date)) if df.genre[x]==g]].toDropMFCC))
    mfccOUT.append((g,kmeans.cluster_centers_))

for x in range(20): #aoooo ma so fino a 20 no a 32, gli mfcc, 1,21 quindi; mi pesa il culo faccio (20)
    df["mfcc{}".format(x)]=0.0


from scipy.spatial import distance
def findTheRightCentroid(mfcc, centroid): #_20, _32_20
    l=[(distance.euclidean(mfcc,centroid[x]),x) for x in range(32)]
    spotMin = min([x[0] for x in l])
    return [centroid[x[1]] for x in l if x[0]==spotMin][0]

for x in range(len(df.date)):
    rightCentroid=findTheRightCentroid(df["toDropMFCC"][x], [y[1].tolist() for y in mfccOUT if df.genre[x]==y[0]][0]) #i pass the row + the right centroidS wrt song genre
    df["mfcc1"][x],df["mfcc2"][x],df["mfcc3"][x],df["mfcc4"][x],df["mfcc5"][x],df["mfcc6"][x],df["mfcc7"][x],df["mfcc8"][x],df["mfcc9"][x],df["mfcc10"][x],df["mfcc11"][x],df["mfcc12"][x],df["mfcc13"][x],df["mfcc14"][x],df["mfcc15"][x],df["mfcc16"][x],df["mfcc17"][x],df["mfcc18"][x],df["mfcc19"][x],df["mfcc0"][x],=rightCentroid
    #df[["mfcc{}".format(x) for x in range(20)]].iloc[x] = rightCentroid #CI METTO I 20 COSI DEL CENTRODIE GIUSTO... BOH

#df.to_excel("AfterDeath15Settembre.xlsx")
df1 = pd.read_excel(r"C:\Users\alessio\Desktop\AfterDeath15Settembre.xlsx", engine="openpyxl")
df[["mfcc{}".format(x) for x in range(20)]] = df1[["mfcc{}".format(x) for x in range(20)]]
#df.to_excel("Rebirth.xlsx")




########################################https://blog.floydhub.com/introduction-to-k-means-clustering-in-python-with-scikit-learn/

for x in range(len(df.date)):
    try:
        el=[y for y in mfccOUT if df.genre[x]==y[0]][0][-1]
        df["Timbre1"][x],df["Timbre2"][x],df["Timbre3"][x],df["Timbre4"][x],df["Timbre5"][x],df["Timbre6"][x]=el
    except:
        print(df.song[x])


df1 = pd.DataFrame(mfcc)
kmeans = KMeans(n_clusters=32, random_state=42).fit(df1)
#kmeans.labels_
mfccOUT.append(kmeans.cluster_centers_)

######################################
# with open(r"C:\Users\alessio\Documents\scTimbreFive.pkl", 'rb') as f:
#    scTimbreOUT = pickle.load(f)

df["Timbre1"],df["Timbre2"],df["Timbre3"],df["Timbre4"],df["Timbre5"],df["Timbre6"]=0.0,0.0,0.0,0.0,0.0,0.0

for x in range(len(df.date)):
    try:
        el=[y for y in scTimbreOUT if y[0]==df.song[x] and y[1]==df.artist[x]][0][-1]
        df["Timbre1"][x],df["Timbre2"][x],df["Timbre3"][x],df["Timbre4"][x],df["Timbre5"][x],df["Timbre6"][x]=el
    except:
        print(df.song[x])
#
# with open(r"C:\Users\alessio\Downloads\done2.pkl", 'rb') as f:
#    chromaOUT = pickle.load(f)

df["Chroma1"],df["Chroma2"],df["Chroma3"],df["Chroma4"],df["Chroma5"],df["Chroma6"]=0.0,0.0,0.0,0.0,0.0,0.0

for x in range(len(df.date)):
    try:
        el=[y for y in scChromaOUT if y[0]==df.song[x] and y[1]==df.artist[x]][0][-1]
        df["Chroma1"][x],df["Chroma2"][x],df["Chroma3"][x],df["Chroma4"][x],df["Chroma5"][x],df["Chroma6"][x]=el
    except:
        print(df.song[x])

df["ArousalMean"],df["ArousalSD"]=0.0,0.0
for x in range(len(df.date)):
    try:
        el = [y for y in arousalOUT if y[0]==df.song[x] and y[1]==df.artist[x]][0]
        df["ArousalMean"][x],df["ArousalSD"][x] = el[2], el[3]
    except:
        print(df.song[x])


######################################################################### 15 settembre testiamo sti cazzo di modelli
import os, librosa, numpy as np, pandas as pd, statistics, scipy
df = pd.read_excel(r"C:\Users\alessio\Desktop\Rebirth.xlsx", engine='openpyxl')
# df = df.sort_values(by="uniq_id")
# df = df.reset_index()
# df = df.drop(columns="index")
# df["Rank"] = 0
# i,j = 0, [x for x in range(50,0,-1)]
# while i<=len(df.genre):
#     df.iloc[i:i+50]["Rank"] = j
#     i+=50

# Chroma alone
import statsmodels.api as sm
my_cols = ["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

# Timbre alone
import statsmodels.api as sm
my_cols = ["Timbre1","Timbre2","Timbre3","Timbre4","Timbre5","Timbre6",]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

# Arousal alone
import statsmodels.api as sm
my_cols = ["ArousalSD","ArousalMean"]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

#Mfcc alone
import statsmodels.api as sm
my_cols = ["mfcc{}".format(x) for x in range(20)] + ["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",] +["ArousalSD","ArousalMean"] + ["Timbre1","Timbre2","Timbre3","Timbre4","Timbre5","Timbre6",]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

###################### 2° parte del Modelling, 16 settembre 2021
import os, librosa, numpy as np, pandas as pd, statistics, scipy
df = pd.read_excel(r"C:\Users\alessio\Desktop\Rebirth.xlsx", engine='openpyxl')
#df[list(df1.columns)[-35:]] = df1[[list(df1.columns)[-35:]][0]]


# Random Forest Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

my_cols= my_cols + ["lsm"]+["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",] +["ArousalSD","ArousalMean"] + ["Timbre1","Timbre2","Timbre3","Timbre4","Timbre5","Timbre6",]
my_cols=my_cols    + list(df.columns)[-1126-1885:-1126]

my_cols.append("Rank")
df1 = df[my_cols]


X_train, X_test, y_train, y_test = train_test_split(df1.drop(columns=["Rank"]), df1["Rank"], test_size=0.33, random_state=42)


rf = RandomForestRegressor(random_state=42, bootstrap= True, max_depth= 10, min_samples_leaf =1, min_samples_split =10, n_estimators =1000)
rf.fit(X_train, y_train)

predictions = rf.predict(X_test)
errors = abs(predictions - y_test)
mean_absolute_error(y_test, predictions)
mean_squared_error(y_test,predictions, squared=True)

# Grid for GridSearchCV
grid_rf = {'n_estimators': [200, 1000],
           'max_depth': [10, 100],
           'min_samples_split': [2, 10],
           'min_samples_leaf': [1, 4],
           'bootstrap': [True, False]}

# GridSearch and Model Fit
something_rf = GridSearchCV(RandomForestRegressor(), param_grid=grid_rf, cv=5, verbose=2)
something_rf.fit(X_train, y_train)
something_rf.best_params_ #{'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 1000}
predictions = something.predict(X_test)
errors = abs(predictions - y_test)

mean_squared_error(y_test, predictions, squared=True)
mean_absolute_error(y_test, predictions)
scores_rf = cross_val_score(
    RandomForestRegressor(random_state=42, n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_depth=100,
                          bootstrap=True), df.drop(columns=["cost"]), df["cost"],
    cv=10)  # df.drop(columns=["cost"]), df["cost"], cv=10)
scores_rf.mean()


#SVMMMMMMMMMMMMMMMMM
from sklearn.preprocessing import StandardScaler
X, y = df1.drop(columns=["Rank"]), df1["Rank"]
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(np.array(y).reshape(-1,1))
#df1=df1.drop(columns="Rank")

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
regressor = SVR(kernel='rbf')
regressor.fit(X_train,y_train)
y_pred = regressor.predict(X_test)
mean_squared_error(y_test, y_pred)
mean_absolute_error(y_test, y_pred)


my_col1 = ["Rank"]+["Chroma1","Chroma2","Chroma3","Chroma4","Chroma5","Chroma6",] +["ArousalSD","ArousalMean"] + ["Timbre1","Timbre2","Timbre3","Timbre4","Timbre5","Timbre6",]
df1 = df[my_col1]

my_cols2=["Rank"]+[x for x in my_cols if x not in my_col1]
df1 = df[my_cols2]

df1 = df[my_cols]

# SHAPPPPPPPPPPPPPPPPPPPPPP
rf = RandomForestRegressor(random_state=42, bootstrap= True, max_depth= 10, min_samples_leaf =1, min_samples_split =10, n_estimators =1000)
rf.fit(X,y)

from matplotlib import pyplot as plt
rf.feature_importances_
plt.barh(list(df1.drop(columns=["Rank"]).columns), rf.feature_importances_)

sorted_idx = rf.feature_importances_.argsort()
plt.barh(np.array(list(df1.drop(columns=["Rank"]).columns))[sorted_idx], rf.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

import shap
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_train)

shap.summary_plot(shap_values, X_train, feature_names=list(df1.drop(columns=["Rank"]).columns))

shap.summary_plot(shap_values, X_train, feature_names=list(df1.drop(columns=["Rank"]).columns), plot_type="bar")

shap.dependence_plot(3, shap_values, X_train, feature_names=list(df1.drop(columns=["Rank"]).columns))

shap.decision_plot(explainer.expected_value[0], shap_values[:100], feature_names = list(df1.drop(columns=["Rank"]).columns))
